# RunnableLambda 객체 사용방법 실습

## 환경변수 로딩

In [19]:
from dotenv import load_dotenv
import os
load_dotenv(override=True, dotenv_path="../.env")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## RunnableLambda 객체 사용방법

In [2]:
from langchain_core.runnables import RunnableLambda

# 함수 정의
def func(x):
    return  x*2


In [3]:
# 함수를 전달인자로 넣기
runnable_1 = RunnableLambda(func)


In [4]:
input_data = 10
# RunnableLambda를 통한 함수 실행
print(runnable_1.invoke(input_data))

20


In [5]:
r1 =RunnableLambda(lambda x: x*2)
r1.invoke(10)

20

In [6]:
# runerable batch
r3 = RunnableLambda(func)
input_data = [10, 20, 30]

r3.batch(input_data)

[20, 40, 60]

## 순차적으로 실행하게 하기

In [ ]:
from langchain_core.runnables import RunnableLambda

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 5)

chain = r1 | r2

chain.invoke(10)

35

In [8]:
from langchain_core.runnables import RunnableLambda, RunnableSequence

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 5)

# chain = r1 | r2  # 아래와 같은 표현
chain = RunnableSequence(r1, r2)

chain.invoke(10)

35

In [9]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 5)

# 병렬처리 , r1, r2 키는 임으로 설정해 주면 됨.
# chain = RunnableParallel(r1=r1, r2=r2)
chain = RunnableParallel(first=r1, second=r2)

chain.invoke(10)

{'first': 30, 'second': 15}

In [10]:
[{"foo":2}]*3

[{'foo': 2}, {'foo': 2}, {'foo': 2}]

In [11]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

runnable_1 = RunnableLambda(lambda x: {"foo": x})
runnable_2 = RunnableLambda(lambda x: [x] * 3)
runnable_3 = RunnableLambda(lambda x: str(x))

chain = runnable_1 | RunnableParallel(r2=runnable_2, r3=runnable_3)

print(chain.invoke(2))
chain.get_graph().print_ascii()

{'r2': [{'foo': 2}, {'foo': 2}, {'foo': 2}], 'r3': "{'foo': 2}"}
        +-------------+        
        | LambdaInput |        
        +-------------+        
               *               
               *               
               *               
          +--------+           
          | Lambda |           
          +--------+           
               *               
               *               
               *               
   +----------------------+    
   | Parallel<r2,r3>Input |    
   +----------------------+    
          *         *          
        **           **        
       *               *       
+--------+          +--------+ 
| Lambda |          | Lambda | 
+--------+          +--------+ 
          *         *          
           **     **           
             *   *             
  +-----------------------+    
  | Parallel<r2,r3>Output |    
  +-----------------------+    


## LCEL 문법 적용

In [33]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", # 현재 안정화된 최신 버전은 1.5 또는 2.0입니다.
    google_api_key=GEMINI_API_KEY,   # 'GEMINI_API_KEY'가 아니라 'google_api_key'입니다.
    temperature=0.1
)


- 방법1

In [34]:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
    다음 컨텍스트에 대해서만 답하세요.
    컨텍스트 :
    {context}
    질문:
    {query}
""")

chain = prompt | llm
input = {
            "context": "Vector Search에 의한 컨텐스트 내용", 
            "query": "안녕"
        }
print(chain.invoke(input))

content="주어진 컨텍스트 내에서만 답변해야 합니다.\n\n컨텍스트는 'Vector Search에 의한 컨텐스트 내용'입니다. 하지만 '안녕'이라는 질문은 해당 컨텍스트와 관련이 없습니다.\n\n따라서 '안녕'이라는 인사에는 직접 답변할 수 없습니다. Vector Search에 대해 궁금한 점이 있으시면 질문해주세요." additional_kwargs={} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019b8d20-27bb-7340-8762-6691de1ad584-0' usage_metadata={'input_tokens': 39, 'output_tokens': 696, 'total_tokens': 735, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 621}}


- 방법2 RunnableLambda 사용

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

prompt = ChatPromptTemplate.from_template("""
    다음 컨텍스트에 대해서만 답하세요.
    컨텍스트 :
    {context}
    질문:
    {query}
""")

# 기본값을 제공하는 함수
def add_default_values(input_dict):
    return {
        "context": input_dict.get("context", "Vector Search에 의한 컨텐스트 내용"),
        "query": input_dict.get("query", "안녕")
    }


# 체인 구성
chain = RunnableLambda(add_default_values) | prompt | llm

In [22]:
# 빈 딕셔너리로도 실행 가능
result = chain.invoke({})
print(result)

content='죄송합니다. 제공해주신 컨텍스트("Vector Search에 의한 컨텐스트 내용")는 기술적인 내용이며, \'안녕\'이라는 질문은 해당 컨텍스트와 관련이 없습니다.\n\n컨텍스트와 관련된 질문만 답변해 드릴 수 있습니다.' additional_kwargs={} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019b8d1c-4ed6-7281-90bd-b5f259a0f477-0' usage_metadata={'input_tokens': 39, 'output_tokens': 202, 'total_tokens': 241, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 146}}


In [15]:
# 또는 일부 값만 override
result2 = chain.invoke({"query": "RunnableLambda에 대해 1문장으로 설명해줘"})

In [16]:
print(result2.content)

제공된 컨텍스트에 한해 답변해야 하므로, 컨텍스트에 'RunnableLambda'에 대한 내용이 없어 해당 질문에 답할 수 없습니다.


## 실행을 그래프로 시각화 하기

In [17]:
chain.get_graph().print_ascii()

  +--------------------------+   
  | add_default_values_input |   
  +--------------------------+   
                *                
                *                
                *                
     +--------------------+      
     | add_default_values |      
     +--------------------+      
                *                
                *                
                *                
     +--------------------+      
     | ChatPromptTemplate |      
     +--------------------+      
                *                
                *                
                *                
   +------------------------+    
   | ChatGoogleGenerativeAI |    
   +------------------------+    
                *                
                *                
                *                
+------------------------------+ 
| ChatGoogleGenerativeAIOutput | 
+------------------------------+ 
